In [1]:
from typing import Optional

import torch
import torch.nn as nn
from transformers import AutoConfig, AutoModel, AutoModelForCausalLM

class RewardModel(nn.Module):
    def __init__(
        self, pretrain_or_model: str, from_config=False, normalize_reward=True, use_flash_attention_2=False, dtype="auto") -> None:
        super().__init__()

        self.model = AutoModelForCausalLM.from_pretrained(
            pretrain_or_model,
            torch_dtype=dtype,
            trust_remote_code=True,
            use_flash_attention_2=use_flash_attention_2,
        ).model
        # value head
        self.value_head = nn.Linear(self.model.config.hidden_size, 1)
        self.value_head.weight.data.normal_(mean=0.0, std=1 / (self.model.config.hidden_size + 1))

        # mean std
        self.normalize_reward = normalize_reward
        self.register_buffer("mean", torch.zeros(1))
        self.register_buffer("std", torch.ones(1))

    def forward(self, sequences: torch.LongTensor, attention_mask: Optional[torch.Tensor] = None) -> torch.Tensor:
        outputs = self.model(sequences, attention_mask=attention_mask)
        last_hidden_states = outputs["last_hidden_state"]
        values = self.value_head(last_hidden_states).squeeze(-1)

        # left padding in training mode
        if self.training:
            reward = values[:, -1]
        else:
            # assume that there is some padding on both sides
            last_value = []
            for i in range(sequences.size(0)):
                for t in reversed(range(sequences.size(1))):
                    if attention_mask[i][t] > 0.5:
                        last_value.append(values[i][t])
                        break
            reward = torch.stack(last_value)

            # normalize reward in eval mode
            if self.normalize_reward:
                reward = (reward - self.mean) / self.std
        return reward

/root/miniconda3/envs/zzd-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import torch
import torch.nn as nn
import os
import json
# os.environ['CUDA_VISIBLE_DEVICES'] = '1'
from transformers import AutoModelForCausalLM, AutoTokenizer
device = 'cuda:1'

base_model = '/data/share_user/zzd/ckpt/Baichuan2-13B-Chat'
ckpt_path = '/data/share_user/zzd/ckpt/rlhf_baichuan2/1124/rm_model.pt'
def init_model():
    print("init model ...")
    model = RewardModel(base_model, dtype=torch.float16)
    print("weight loading ...")

    model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
    model.half().eval().to(device)
    
    tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=False, trust_remote_code=True)
    tokenizer.padding_side = 'left'
    return model, tokenizer

model, tokenizer = init_model()


init model ...


Loading checkpoint shards: 100%|██████████| 3/3 [00:18<00:00,  6.19s/it]


weight loading ...


In [3]:
class MyDataset:
    """Dataset for supervised fine-tuning."""
    def __init__(self, tokenizer, model_max_length=4096, user_tokens=[195], assistant_tokens=[196],):
        self.tokenizer = tokenizer
        self.model_max_length = model_max_length
        self.user_tokens = user_tokens
        self.assistant_tokens = assistant_tokens
        self.ignore_index = -100

    def __len__(self):
        return len(self.data)

    def preprocessing(self, messages):
        input_ids = []
        labels = []

        for message in messages:
            from_ = message["role"]
            value = message["content"]
            value_ids = self.tokenizer.encode(value)

            if from_ == "user":
                input_ids += self.user_tokens + value_ids
                labels += [self.tokenizer.eos_token_id] + [self.ignore_index] * len(value_ids)
            else:
                input_ids += self.assistant_tokens + value_ids
                labels += [self.ignore_index] + value_ids
        input_ids.append(self.tokenizer.eos_token_id)
        labels.append(self.tokenizer.eos_token_id)
        input_ids = input_ids[: self.model_max_length]
        labels = labels[: self.model_max_length]
        input_ids += [self.tokenizer.pad_token_id] * (self.model_max_length - len(input_ids))
        labels += [self.ignore_index] * (self.model_max_length - len(labels))
        input_ids = torch.LongTensor(input_ids)
        labels = torch.LongTensor(labels)
        attention_mask = input_ids.ne(self.tokenizer.pad_token_id)
        return input_ids, labels, attention_mask

dataset = MyDataset(tokenizer)

In [4]:
file_name = 'youxiake'

import pandas as pd
from tqdm import tqdm

df = pd.read_excel(f'{file_name}_tmp.xlsx', sheet_name='Sheet1')

model_list = ['sft[1]', 'gpt4[2]', 'Yi-34B[3]', 'Aquila-34B[4]', 'deepseek-67B[5]']

df['score'] = 0
for i in tqdm(range(len(df))):
    score_map = {'sft[1]':0.0, 'gpt4[2]':0.0, 'Yi-34B[3]':0.0, 'Aquila-34B[4]':0.0, 'deepseek-67B[5]':0.0}
    for model_name in model_list:
        messages = [{"role": "user", "content": df['query'][i]}, {'role': 'assistant','content':df[model_name][i]}]
        input_ids, labels, attention_mask = dataset.preprocessing(messages)
        # print(input_ids, attention_mask)
        torch.cuda.empty_cache()
        with torch.no_grad():
            reward = model(input_ids.unsqueeze(0).to(device), attention_mask.unsqueeze(0).to(device))
        score_map[model_name] = reward.detach().cpu().numpy()[0]
    df['score'][i] = score_map

save_path = f'{file_name}_score.xlsx'
writer = pd.ExcelWriter(save_path)
df.to_excel(writer, sheet_name='Sheet1')
writer.close()

  0%|          | 0/150 [00:00<?, ?it/s]/tmp/ipykernel_529225/2859308612.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['score'][i] = score_map
/tmp/ipykernel_529225/2859308612.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '{'sft[1]': 1.253, 'gpt4[2]': 1.23, 'Yi-34B[3]': 1.264, 'Aquila-34B[4]': 1.27, 'deepseek-67B[5]': 1.318}' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df['score'][i] = score_map
100%|██████████| 150/150 [12:45<00:00,  5.11s/it]


In [5]:
model

RewardModel(
  (model): BaichuanModel(
    (embed_tokens): Embedding(125696, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x BaichuanLayer(
        (self_attn): BaichuanAttention(
          (W_pack): Linear(in_features=5120, out_features=15360, bias=False)
          (o_proj): Linear(in_features=5120, out_features=5120, bias=False)
        )
        (mlp): MLP(
          (gate_proj): Linear(in_features=5120, out_features=13696, bias=False)
          (down_proj): Linear(in_features=13696, out_features=5120, bias=False)
          (up_proj): Linear(in_features=5120, out_features=13696, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): RMSNorm()
        (post_attention_layernorm): RMSNorm()
      )
    )
    (norm): RMSNorm()
  )
  (value_head): Linear(in_features=5120, out_features=1, bias=True)
)